In [1]:
import pandas as pd

In [29]:
import os

print(os.getcwd())      # dossier de travail actuel
print(os.listdir())     # fichiers que Python voit dans ce dossier


c:\Users\33664\OneDrive\Bureau\Auditez un environnement de données
['Copie+de+Extraction+cube+OLAP+-+14+aout+2024 (1).xlsx', 'ex1.ipynb', 'je veux un seul pour tout les tables.xlsx', 'Template+de+support+de+présentation+.pptx', 'Untitled.pdf']


### 📂 Chargement des données de ventes immobilières

In [2]:
import pandas as pd

fichier = pd.read_excel("Copie+de+Extraction+cube+OLAP+-+14+aout+2024 (1).xlsx")

# Afficher seulement 2 lignes
print(fichier.head(2))


                                ID_BDD        CUSTOMER_ID  \
0  HZDG8U15NNY7SI6HDK8NMFDEK7MOVUX31VY  CUST-G42Z6WE8QLWJ   
1  1H51BRR800TK9DCIH8M9QCRH3LEAR0JWALJ  CUST-CUA37GP8GABQ   

                         id_employe            EAN  Date achat ID ticket  
0  b413ca065a762e8cf2e86cfea8b9c174  6473630445822       45518    t_2693  
1  a7ada0770091e838e3dcd45265282820  1857802002765       45518    t_4408  


In [8]:
import pandas as pd
import sqlite3

# 📂 Charger le fichier Excel
fichier = "Copie+de+Extraction+cube+OLAP+-+14+aout+2024 (1).xlsx"

# Vérifier les feuilles disponibles
xls = pd.ExcelFile(fichier)
print("📑 Feuilles disponibles :", xls.sheet_names)

# ✅ Utilisation de get() pour éviter les erreurs si la feuille n'existe pas
def lire_feuille(nom):
    if nom in xls.sheet_names:
        return pd.read_excel(fichier, sheet_name=nom)
    else:
        print(f"⚠️ Feuille '{nom}' introuvable, sautée.")
        return pd.DataFrame()

# Lire les feuilles
clients_raw = lire_feuille("Clients")
produits_raw = lire_feuille("Produits")
employes_raw = lire_feuille("Employé")
calendrier_raw = lire_feuille("Calendrier")
ventes_raw = lire_feuille("Vente_Detail")  # adapte selon le vrai nom affiché par print

# 🗂 Table Clients
if not clients_raw.empty:
    clients = clients_raw.rename(columns={
        "CUSTOMER_ID": "CUSTOMER_ID",
        "date_inscription": "date_inscription"
    })

# 🗂 Table Produits
if not produits_raw.empty:
    produits = produits_raw.rename(columns={
        "EAN": "EAN",
        "categorie": "categorie",
        "rayon": "rayon",
        "libelle_produit": "libelle_produit",
        "prix": "prix"
    })

# 🗂 Table Employes
if not employes_raw.empty:
    employes = employes_raw.rename(columns={
        "id_employe": "id_employe",
        "employe": "employe",
        "prenom": "prenom",
        "nom": "nom",
        "date_debut": "date_debut",
        "hash_mdp": "hash_mdp",
        "mail": "mail"
    })

# 🗂 Table Calendrier
if not calendrier_raw.empty:
    calendrier = calendrier_raw.rename(columns={
        "date": "date",
        "annee": "annee",
        "mois": "mois",
        "jour": "jour",
        "mois_nom": "mois_nom",
        "annee_mois": "annee_mois",
        "jour_semaine": "jour_semaine",
        "trimestre": "trimestre"
    })

# 🗂 Table Vente_Detail
if not ventes_raw.empty:
    ventes = ventes_raw.rename(columns={
        "ID_BDD": "ID_BDD",
        "CUSTOMER_ID": "CUSTOMER_ID",
        "id_employe": "id_employe",
        "EAN": "EAN",
        "Date_achat": "Date_achat",
        "ID_ticket": "ID_ticket"
    })

# 💾 Connexion SQLite
conn = sqlite3.connect("ventes.sqlite")

# 📥 Sauvegarde des tables (seulement si elles existent)
if not clients_raw.empty:
    clients.to_sql("Clients", conn, if_exists="replace", index=False)
if not produits_raw.empty:
    produits.to_sql("Produits", conn, if_exists="replace", index=False)
if not employes_raw.empty:
    employes.to_sql("Employes", conn, if_exists="replace", index=False)
if not calendrier_raw.empty:
    calendrier.to_sql("Calendrier", conn, if_exists="replace", index=False)
if not ventes_raw.empty:
    ventes.to_sql("Vente_Detail", conn, if_exists="replace", index=False)

# 🔒 Fermeture
conn.close()
print("✅ Base SQLite créée avec succès : ventes.sqlite")


📑 Feuilles disponibles : ['Vente Détail', 'Produits', 'Clients', 'Calendrier', 'Employé']
⚠️ Feuille 'Vente_Detail' introuvable, sautée.
✅ Base SQLite créée avec succès : ventes.sqlite


In [1]:
import pandas as pd
import sqlite3
import re

# 📂 Charger le fichier Excel
fichier = "Copie+de+Extraction+cube+OLAP+-+14+aout+2024 (1).xlsx"
xls = pd.ExcelFile(fichier)
print("📑 Feuilles disponibles :", xls.sheet_names)

# 💾 Connexion à SQLite
conn = sqlite3.connect("ventes.sqlite")

# 🔄 Boucle sur chaque feuille
for feuille in xls.sheet_names:
    print(f"➡️ Import de la feuille : {feuille}")
    
    # Charger la feuille en DataFrame
    df = pd.read_excel(fichier, sheet_name=feuille)
    
    if df.empty:
        print(f"⚠️ Feuille '{feuille}' vide, sautée.")
        continue
    
    # Si la feuille contient une colonne "date" en format Excel, convertir en datetime
    if 'date' in df.columns:
        df['date'] = pd.to_timedelta(df['date'], unit='D') + pd.Timestamp('1899-12-30')
    
    # Nettoyer le nom de la table pour SQLite (remplacer les espaces par _)
    table_name = feuille.replace(" ", "_")
    
    # Sauvegarder dans SQLite
    df.to_sql(table_name, conn, if_exists="replace", index=False)
    print(f"   ✅ Table '{table_name}' créée ({len(df)} lignes)")

# 🔒 Fermeture
conn.close()
print("🎉 Import terminé ! Toutes les feuilles Excel sont dans ventes.sqlite")


📑 Feuilles disponibles : ['Vente Détail', 'Produits', 'Clients', 'Calendrier', 'Employé']
➡️ Import de la feuille : Vente Détail
   ✅ Table 'Vente_Détail' créée (41377 lignes)
➡️ Import de la feuille : Produits
   ✅ Table 'Produits' créée (18040 lignes)
➡️ Import de la feuille : Clients
   ✅ Table 'Clients' créée (2297 lignes)
➡️ Import de la feuille : Calendrier
   ✅ Table 'Calendrier' créée (1999 lignes)
➡️ Import de la feuille : Employé
   ✅ Table 'Employé' créée (56 lignes)
🎉 Import terminé ! Toutes les feuilles Excel sont dans ventes.sqlite


In [22]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("ventes.sqlite")

# Charger la table Vente_Détail
ventes = pd.read_sql('SELECT * FROM "Vente_Détail"', conn)

# Convertir la colonne "Date achat" depuis Excel (numérique) en datetime
ventes['Date achat'] = pd.to_timedelta(ventes['Date achat'], unit='D') + pd.Timestamp('1899-12-30')

# Réinjecter la table dans SQLite
ventes.to_sql("Vente_Détail", conn, if_exists="replace", index=False)

conn.close()
print("✅ Conversion de la colonne 'Date achat' terminée")


✅ Conversion de la colonne 'Date achat' terminée


In [10]:
['Clients', 'Produits', 'Employé', 'Calendrier', 'Vente Detail']


['Clients', 'Produits', 'Employé', 'Calendrier', 'Vente Detail']

In [28]:
import pandas as pd
import sqlite3

# Connexion SQLite
conn = sqlite3.connect("ventes.sqlite")

# -------------------------------
# Vente_Détail
# -------------------------------
ventes = pd.read_sql('SELECT * FROM "Vente_Détail"', conn)

if pd.api.types.is_numeric_dtype(ventes['Date achat']):
    ventes['Date achat'] = pd.to_timedelta(ventes['Date achat'], unit='D') + pd.Timestamp('1899-12-30')

ventes['Date achat'] = pd.to_datetime(ventes['Date achat']).dt.strftime('%Y-%m-%d')
ventes.to_sql("Vente_Détail", conn, if_exists="replace", index=False)

# -------------------------------
# Calendrier
# -------------------------------
calendrier = pd.read_sql('SELECT * FROM Calendrier', conn)

if pd.api.types.is_numeric_dtype(calendrier['date']):
    calendrier['date'] = pd.to_timedelta(calendrier['date'], unit='D') + pd.Timestamp('1899-12-30')

calendrier['date'] = pd.to_datetime(calendrier['date']).dt.strftime('%Y-%m-%d')
calendrier.to_sql("Calendrier", conn, if_exists="replace", index=False)

conn.close()
print("✅ Dates formatées et tables réinjectées correctement")


✅ Dates formatées et tables réinjectées correctement


In [1]:
import sqlite3

conn = sqlite3.connect("ventes.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('Employes',), ('Produits',), ('Clients',), ('Employé',), ('Vente_Détail',), ('Calendrier',)]


In [4]:
import pandas as pd

try:
    # Charge le fichier Excel dans un DataFrame
    logs_df = pd.read_excel('logs.xlsx')

    # Affiche les premières lignes pour vérifier que le chargement a fonctionné
    print("Aperçu des données du fichier logs.xlsx :")
    print(logs_df.head(7))

except FileNotFoundError:
    print("Erreur : Le fichier 'logs.xlsx' n'a pas été trouvé. Assurez-vous que le fichier est dans le même dossier que ce script.")
except Exception as e:
    print(f"Une erreur est survenue lors du chargement du fichier : {e}")

Aperçu des données du fichier logs.xlsx :
                            id_user   date  action table_insert  \
0  0859b70a9ca905a870bfccc5f8440ba1  45518  INSERT       Client   
1  c4f0909120cfde7086a3c1a56e96a015  45518  INSERT       Client   
2  dd595f0f0b3400df2908f0be7723dad4  45518  INSERT       Client   
3  e44eabec860498a79e675200bc8fa455  45518  INSERT       Client   
4  37c6a856b2e14217855d808fbc8e56a7  45518  INSERT       Client   

            id_ligne            champs               detail  Unnamed: 7  
0  CUST-0P9K9MICI4V3  date_inscription  2024-08-14 00:00:00         NaN  
1  CUST-SZ9IS3Q1LBEP  date_inscription  2024-08-14 00:00:00         NaN  
2  CUST-L99QEXFV9WTS  date_inscription  2024-08-14 00:00:00         NaN  
3  CUST-9SBDRZ728025  date_inscription  2024-08-14 00:00:00         NaN  
4  CUST-3F47X2KHYXQM  date_inscription  2024-08-14 00:00:00         NaN  


In [5]:
import pandas as pd
import sqlite3

# Nom du fichier de logs
fichier_logs = "logs.xlsx"

try:
    # Charger les logs dans un DataFrame
    logs_df = pd.read_excel(fichier_logs)

    # Convertir la colonne 'date' du format numérique Excel en format de date standard
    logs_df['date'] = pd.to_datetime(logs_df['date'], unit='D', origin='1899-12-30')

    # Supprimer la colonne inutile si elle existe
    if 'Unnamed: 7' in logs_df.columns:
        logs_df = logs_df.drop('Unnamed: 7', axis=1)

    # Connexion à la base de données SQLite
    conn = sqlite3.connect("ventes.sqlite")

    # Insérer le DataFrame propre dans une nouvelle table 'Logs'
    logs_df.to_sql("Logs", conn, if_exists="replace", index=False)

    print("✅ Intégration des logs réussie !")
    print("\n🔎 Aperçu des 5 premières lignes de la table 'Logs' après l'intégration :")
    verification_df = pd.read_sql_query("SELECT * FROM Logs LIMIT 5", conn)
    print(verification_df)

except FileNotFoundError:
    print("⚠️ Erreur : Le fichier 'logs.xlsx' est introuvable. Assurez-vous qu'il se trouve dans le même répertoire que ce script.")
except Exception as e:
    print(f"❌ Une erreur s'est produite lors de l'exécution : {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

✅ Intégration des logs réussie !

🔎 Aperçu des 5 premières lignes de la table 'Logs' après l'intégration :
                            id_user                 date  action table_insert  \
0  0859b70a9ca905a870bfccc5f8440ba1  2024-08-14 00:00:00  INSERT       Client   
1  c4f0909120cfde7086a3c1a56e96a015  2024-08-14 00:00:00  INSERT       Client   
2  dd595f0f0b3400df2908f0be7723dad4  2024-08-14 00:00:00  INSERT       Client   
3  e44eabec860498a79e675200bc8fa455  2024-08-14 00:00:00  INSERT       Client   
4  37c6a856b2e14217855d808fbc8e56a7  2024-08-14 00:00:00  INSERT       Client   

            id_ligne            champs               detail  
0  CUST-0P9K9MICI4V3  date_inscription  2024-08-14 00:00:00  
1  CUST-SZ9IS3Q1LBEP  date_inscription  2024-08-14 00:00:00  
2  CUST-L99QEXFV9WTS  date_inscription  2024-08-14 00:00:00  
3  CUST-9SBDRZ728025  date_inscription  2024-08-14 00:00:00  
4  CUST-3F47X2KHYXQM  date_inscription  2024-08-14 00:00:00  
